In [1]:
import pandas as pd
from tqdm import tqdm
import os
import cv2
import numpy as np
import time
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

In [4]:
INPUTDIR = "original_data/train_annots/"
FILENAME = "annotations.csv"


df = pd.DataFrame(columns=['filename', 'class', 'truncated', 'occluded', 'observation angle', \
                           'xmin', 'ymin', 'xmax', 'ymax', 'height', 'width', 'length', \

                           'xloc', 'yloc', 'zloc', 'rot_y'])
def assign_values(filename, idx, list_to_assign):
    df.at[idx, 'filename'] = filename

    df.at[idx, 'class'] = list_to_assign[0]
    df.at[idx, 'truncated'] = list_to_assign[1]
    df.at[idx, 'occluded'] = list_to_assign[2]
    df.at[idx, 'observation angle'] = list_to_assign[3]

    # bbox coordinates
    df.at[idx, 'xmin'] = list_to_assign[4]
    df.at[idx, 'ymin'] = list_to_assign[5]
    df.at[idx, 'xmax'] = list_to_assign[6]
    df.at[idx, 'ymax'] = list_to_assign[7]

    # 3D object dimensions
    df.at[idx, 'height'] = list_to_assign[8]
    df.at[idx, 'width'] = list_to_assign[9]
    df.at[idx, 'length'] = list_to_assign[10]

    # 3D object location
    df.at[idx, 'xloc'] = list_to_assign[11]
    df.at[idx, 'yloc'] = list_to_assign[12]
    df.at[idx, 'zloc'] = list_to_assign[13]

    # rotation around y-axis in camera coordinates
    df.at[idx, 'rot_y'] = list_to_assign[14]

all_files = sorted(os.listdir(INPUTDIR))
pbar = tqdm(total=len(all_files), position=1)

count = 0
for idx, f in enumerate(all_files):
    pbar.update(1)
    file_object = open(INPUTDIR + f, 'r')
    file_content = [x.strip() for x in file_object.readlines()]

    for line in file_content:
        elements = line.split()
        if elements[0] == 'DontCare':
            continue

        assign_values(f, count, elements)
        count += 1

df.to_csv(FILENAME, index=False)


 12%|█████████▋                                                                     | 915/7481 [00:07<01:10, 92.81it/s]


 20%|███████████████▍                                                              | 1477/7481 [00:15<02:01, 49.46it/s]


 25%|███████████████████▊                                                          | 1897/7481 [00:24<02:15, 41.36it/s]


 30%|███████████████████████▍                                                      | 2249/7481 [00:32<01:42, 51.08it/s]


 34%|██████████████████████████▊                                                   | 2575/7481 [00:41<02:19, 35.26it/s]


...

 93%|████████████████████████████████████████████████████████████████████████▏     | 6927/7481 [04:02<00:32, 16.98it/s]


 95%|█████████████████████████████████████████████████████████████████████████▉    | 7087/7481 [04:12<00:19, 20.66it/s]


 97%|███████████████████████████████████████████████████████████████████████████▋  | 7258/7481 [04:23<00:16, 13.65it/s]


 99%|█████████████████████████████████████████████████████████████████████████████▍| 7432/7481 [04:35<00:04, 10.92it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 7481/7481 [04:38<00:00, 15.37it/s]

In [6]:
df = pd.read_csv('annotations.csv')
new_df = df.loc[df['class'] != 'DontCare']
result_df = pd.DataFrame(columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax',
                           'angle', 'xloc', 'yloc', 'zloc'])
print(new_df)
pbar = tqdm(total=new_df.shape[0], position=1)

for idx, row in new_df.iterrows():
    pbar.update(1)
    if os.path.exists(os.path.join("original_data/train_annots", row['filename'])):
        result_df.at[idx, 'filename'] = row['filename']

        result_df.at[idx, 'xmin'] = int(row['xmin'])
        result_df.at[idx, 'ymin'] = int(row['ymin'])
        result_df.at[idx, 'xmax'] = int(row['xmax'])
        result_df.at[idx, 'ymax'] = int(row['ymax'])

        result_df.at[idx, 'angle'] = row['observation angle']
        result_df.at[idx, 'xloc'] = int(row['xloc'])
        result_df.at[idx, 'yloc'] = int(row['yloc'])
        result_df.at[idx, 'zloc'] = int(row['zloc'])
    else: print('false')

mask = np.random.rand(len(result_df)) < 0.8
train = result_df[mask]
test = result_df[~mask]
print(train)

train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)


  0%|▏                                                                             | 85/40570 [00:00<00:47, 843.65it/s]

         filename       class  truncated  occluded  observation angle    xmin  \
0      000000.txt  Pedestrian        0.0         0              -0.20  712.40   
1      000001.txt       Truck        0.0         0              -1.57  599.41   
2      000001.txt         Car        0.0         0               1.85  387.63   
3      000001.txt     Cyclist        0.0         3              -1.65  676.60   
4      000002.txt        Misc        0.0         0              -1.82  804.79   
...           ...         ...        ...       ...                ...     ...   
40565  007480.txt         Car        0.0         1              -2.90  344.07   
40566  007480.txt         Car        0.0         2              -2.91  382.70   
40567  007480.txt         Car        0.0         0              -1.69  631.71   
40568  007480.txt         Car        0.0         2              -1.62  619.92   
40569  007480.txt         Van        0.0         2              -1.58  605.20   

         ymin    xmax    ym


 13%|█████████▊                                                                  | 5252/40570 [00:07<01:01, 578.66it/s]


 23%|█████████████████▍                                                          | 9286/40570 [00:13<00:57, 542.93it/s]


 31%|███████████████████████▍                                                   | 12665/40570 [00:20<01:03, 442.13it/s]


...

 93%|█████████████████████████████████████████████████████████████████████▌     | 37660/40570 [02:11<00:17, 168.94it/s]


 96%|███████████████████████████████████████████████████████████████████████▋   | 38791/40570 [02:17<00:10, 162.98it/s]


 98%|█████████████████████████████████████████████████████████████████████████▊ | 39909/40570 [02:24<00:04, 157.30it/s]


100%|██████████████████████████████████████████████████████████████████████████▉| 40562/40570 [02:28<00:00, 148.44it/s]

         filename xmin ymin xmax ymax angle xloc yloc zloc
0      000000.txt  712  143  810  307  -0.2    1    1    8
1      000001.txt  599  156  629  189 -1.57    0    1   69
2      000001.txt  387  181  423  203  1.85  -16    2   58
3      000001.txt  676  163  688  193 -1.65    4    1   45
4      000002.txt  804  167  995  327 -1.82    3    1    8
...           ...  ...  ...  ...  ...   ...  ...  ...  ...
40565  007480.txt  344  180  479  224  -2.9   -7    1   26
40566  007480.txt  382  179  495  219 -2.91   -6    1   28
40567  007480.txt  631  179  678  214 -1.69    1    1   31
40568  007480.txt  619  176  647  199 -1.62    1    1   47
40569  007480.txt  605  168  621  187 -1.58    0    1   93

[32363 rows x 9 columns]


In [7]:
# ----------- import data and scaling ----------- #
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
print(df_train)

X_train = df_train[['xmin', 'ymin', 'xmax', 'ymax']].values
y_train = df_train[['zloc']].values

# standardized data
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
y_train = scalar.fit_transform(y_train)

# ----------- create model ----------- #
model = Sequential()
model.add(Dense(16, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, kernel_initializer='normal', activation='relu'))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mean_squared_error', optimizer='adam')

# ----------- define callbacks ----------- #
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
modelname = "model@{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(modelname))

# ----------- start training ----------- #
history = model.fit(X_train, y_train, validation_split=0.1, epochs=5000, batch_size=2048, callbacks=[tensorboard], verbose=1)


100%|███████████████████████████████████████████████████████████████████████████| 40570/40570 [05:22<00:00, 125.63it/s]

         filename  xmin  ymin  xmax  ymax  angle  xloc  yloc  zloc
0      000000.txt   712   143   810   307  -0.20     1     1     8
1      000001.txt   599   156   629   189  -1.57     0     1    69
2      000001.txt   387   181   423   203   1.85   -16     2    58
3      000001.txt   676   163   688   193  -1.65     4     1    45
4      000002.txt   804   167   995   327  -1.82     3     1     8
...           ...   ...   ...   ...   ...    ...   ...   ...   ...
32358  007480.txt   344   180   479   224  -2.90    -7     1    26
32359  007480.txt   382   179   495   219  -2.91    -6     1    28
32360  007480.txt   631   179   678   214  -1.69     1     1    31
32361  007480.txt   619   176   647   199  -1.62     1     1    47
32362  007480.txt   605   168   621   187  -1.58     0     1    93

[32363 rows x 9 columns]
Epoch 1/5000


15/15 [==============================] - 1s 51ms/step - loss: 1.0016 - val_loss: 1.0171
Epoch 2/5000
15/15 [==============================] - 0s 9ms/step - loss: 1.0029 - val_loss: 1.0142
Epoch 3/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.9805 - val_loss: 1.0059
Epoch 4/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.9880 - val_loss: 0.9864
Epoch 5/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.9549 - val_loss: 0.9470
Epoch 6/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.9100 - val_loss: 0.8779
Epoch 7/5000
15/15 [==============================] - 0s 12ms/step - loss: 0.8328 - val_loss: 0.7767
Epoch 8/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.7373 - val_loss: 0.6645
Epoch 9/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.6249 - val_loss: 0.5834
Epoch 10/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.5630 - val_loss: 0.5437
Epoch 11/5

15/15 [==============================] - 0s 7ms/step - loss: 0.0942 - val_loss: 0.0845
Epoch 83/5000
15/15 [==============================] - 0s 6ms/step - loss: 0.0923 - val_loss: 0.0843
Epoch 84/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0892 - val_loss: 0.0843
Epoch 85/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0922 - val_loss: 0.0837
Epoch 86/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0893 - val_loss: 0.0837
Epoch 87/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0869 - val_loss: 0.0837
Epoch 88/5000
15/15 [==============================] - 0s 6ms/step - loss: 0.0867 - val_loss: 0.0837
Epoch 89/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0906 - val_loss: 0.0831
Epoch 90/5000
15/15 [==============================] - 0s 6ms/step - loss: 0.0900 - val_loss: 0.0833
Epoch 91/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0904 - val_loss: 0.0828
Ep


100%|███████████████████████████████████████████████████████████████████████████| 40570/40570 [02:48<00:00, 148.44it/s]

...

15/15 [==============================] - 0s 8ms/step - loss: 0.0595 - val_loss: 0.0600
Epoch 4617/5000
15/15 [==============================] - 0s 12ms/step - loss: 0.0604 - val_loss: 0.0599
Epoch 4618/5000
15/15 [==============================] - 0s 5ms/step - loss: 0.0568 - val_loss: 0.0599
Epoch 4619/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0591 - val_loss: 0.0603
Epoch 4620/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0576 - val_loss: 0.0600
Epoch 4621/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0595 - val_loss: 0.0601
Epoch 4622/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0591 - val_loss: 0.0618
Epoch 4623/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0602 - val_loss: 0.0599
Epoch 4624/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0572 - val_loss: 0.0599
Epoch 4625/5000
15/15 [==============================] - 0s 5ms/step - loss: 0.0623 

15/15 [==============================] - 0s 10ms/step - loss: 0.0581 - val_loss: 0.0596
Epoch 4696/5000
15/15 [==============================] - 0s 12ms/step - loss: 0.0591 - val_loss: 0.0606
Epoch 4697/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0560 - val_loss: 0.0599
Epoch 4698/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0608 - val_loss: 0.0611
Epoch 4699/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0598 - val_loss: 0.0596
Epoch 4700/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0577 - val_loss: 0.0599
Epoch 4701/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0598 - val_loss: 0.0604
Epoch 4702/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0596 - val_loss: 0.0619
Epoch 4703/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0570 - val_loss: 0.0600
Epoch 4704/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0593 

15/15 [==============================] - 0s 7ms/step - loss: 0.0555 - val_loss: 0.0601
Epoch 4775/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0570 - val_loss: 0.0610
Epoch 4776/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0563 - val_loss: 0.0612
Epoch 4777/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0570 - val_loss: 0.0594
Epoch 4778/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0580 - val_loss: 0.0593
Epoch 4779/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0583 - val_loss: 0.0604
Epoch 4780/5000
15/15 [==============================] - 0s 6ms/step - loss: 0.0580 - val_loss: 0.0617
Epoch 4781/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0614 - val_loss: 0.0598
Epoch 4782/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0560 - val_loss: 0.0592
Epoch 4783/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0572 - val

15/15 [==============================] - 0s 7ms/step - loss: 0.0611 - val_loss: 0.0587
Epoch 4854/5000
15/15 [==============================] - 0s 6ms/step - loss: 0.0609 - val_loss: 0.0590
Epoch 4855/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0582 - val_loss: 0.0589
Epoch 4856/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0574 - val_loss: 0.0603
Epoch 4857/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0566 - val_loss: 0.0589
Epoch 4858/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0567 - val_loss: 0.0588
Epoch 4859/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0574 - val_loss: 0.0593
Epoch 4860/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0586 - val_loss: 0.0588
Epoch 4861/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0575 - val_loss: 0.0621
Epoch 4862/5000
15/15 [==============================] - 0s 6ms/step - loss: 0.0598 - 

15/15 [==============================] - 0s 7ms/step - loss: 0.0570 - val_loss: 0.0592
Epoch 4933/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0592 - val_loss: 0.0595
Epoch 4934/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0579 - val_loss: 0.0596
Epoch 4935/5000
15/15 [==============================] - 0s 7ms/step - loss: 0.0584 - val_loss: 0.0593
Epoch 4936/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0565 - val_loss: 0.0588
Epoch 4937/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0573 - val_loss: 0.0592
Epoch 4938/5000
15/15 [==============================] - 0s 10ms/step - loss: 0.0589 - val_loss: 0.0588
Epoch 4939/5000
15/15 [==============================] - 0s 11ms/step - loss: 0.0577 - val_loss: 0.0596
Epoch 4940/5000
15/15 [==============================] - 0s 8ms/step - loss: 0.0579 - val_loss: 0.0589
Epoch 4941/5000
15/15 [==============================] - 0s 9ms/step - loss: 0.0584 -

In [8]:
# ----------- save model and weights ----------- #
model_json = model.to_json()
with open("generated_files/{}.json".format("mymodel"), "w") as json_file:
    json_file.write(model_json)

model.save_weights("generated_files/{}.h5".format("mymodel"))
print("Saved model to disk")

Saved model to disk


In [10]:
# get data
df_test = pd.read_csv('data/test.csv')
X_test = df_test[['xmin', 'ymin', 'xmax', 'ymax']].values
y_test = df_test[['zloc']].values

    # standardized data
scalar = StandardScaler()
X_test = scalar.fit_transform(X_test)
y_test = scalar.fit_transform(y_test)

# load json and create model
json_file = open('generated_files/{}.json'.format("mymodel"), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json( loaded_model_json )

# load weights into new model
loaded_model.load_weights("generated_files/{}.h5".format("mymodel"))
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='adam')
y_pred = loaded_model.predict(X_test)

# scale up predictions to original values
y_pred = scalar.inverse_transform(y_pred)
y_test = scalar.inverse_transform(y_test)

# save predictions
df_result = df_test
df_result['zloc_pred'] = -100000

for idx, row in df_result.iterrows():
    df_result.at[idx, 'zloc_pred'] = y_pred[idx]

df_result.to_csv('data/predictions.csv', index=False)

Loaded model from disk


In [ ]:
df = pd.read_csv("distance-estimator/data/predictions.csv")

for idx, row in df.iterrows():
    if os.path.exists(os.path.join("original_data/train_annots/", row['filename'])):
        fp = os.path.join("original_data/train_images", row['filename'].replace('.txt', '.png'))
        im = cv2.imread(fp)

        x1 = int(row['xmin'])
        y1 = int(row['ymin'])
        x2 = int(row['xmax'])
        y2 = int(row['ymax'])

        cv2.line(im, (int(1224/2), 0), (int(1224/2), 370), (255,255,255), 2)
        cv2.rectangle(im, (x1, y1), (x2, y2), (0, 255, 0), 3)
        string = "(act: {}, pred: {})".format(row['zloc'], row['zloc_pred'])
        cv2.putText(im, string, (int((x1+x2)/2), int((y1+y2)/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

        cv2.imshow("detections", im)
        cv2.waitKey(0)